# Titanic: Survival Model

Build and train a model to predict survival on the Titanic using a [cleaned and split dataset](https://huggingface.co/datasets/jamieoliver/titanic-2410), and upload the model to Hugging Face.

Based on https://github.com/fastai/course22/blob/master/clean/05-linear-model-and-neural-net-from-scratch.ipynb using the dataset from https://www.kaggle.com/competitions/titanic.

Plan
- [ ] Download [cleaned and split dataset](https://huggingface.co/datasets/jamieoliver/titanic-2410) from Hugging Face
- [ ] Prepare data for model
- [ ] Train model
- [ ] Test model
- [ ] Upload model to Hugging Face